In [17]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [18]:
from datetime import datetime

In [19]:
results = []

month_map = {'Jan':'01','Feb':'02','Mar':'03','Apr':'04','May':'05','Jun':'06','Jul':'07','Aug':'08','Sep':'09',
             'Oct':'10','Nov':'11','Dec':'12'}

data_history = {
    "Date": [],
    "source": [],
    "sentiment":[]
}
#stocknewsapi has LLM -GPT4 analyzed sentiment --> which categorizes the news into the buckets positive, negative, neutral
for i in range (1,100):#max10000 data 
    api_url = "https://stocknewsapi.com/api/v1?tickers=AMZN&data=yeartodate&items=100&page="+ str(i) +"&token=metobjbevvotlrizynavabhyisukldmxcxsdwgrf"
    response = requests.get(api_url)
    response_data = response.json()
    for rd in response_data["data"]:
        date_text = rd["date"].split(' ')
        dt = date_text[3] + month_map[date_text[2]] + date_text[1]
        data_history["Date"].append(pd.Timestamp(dt))
        data_history["source"].append(rd["source_name"])
        data_history["sentiment"].append(rd["sentiment"])
    results += response_data["data"]
import pandas as pd
df = pd.DataFrame(data_history)
print(df)

           Date           source sentiment
0    2023-08-06    InvestorPlace  Positive
1    2023-08-06           Forbes   Neutral
2    2023-08-06    Seeking Alpha  Positive
3    2023-08-06  The Motley Fool  Positive
4    2023-08-06  The Motley Fool  Positive
...         ...              ...       ...
9895 2021-07-13          Barrons  Negative
9896 2021-07-13    Yahoo Finance  Positive
9897 2021-07-13  The Motley Fool  Positive
9898 2021-07-13     Market Watch  Negative
9899 2021-07-13          Reuters  Positive

[9900 rows x 3 columns]


In [49]:
# fetch the derivatives data (this decides the volatility and the vix index)--> will integrate it today

In [20]:
df['sentiment'] = df['sentiment'].map({'Positive': 1, 'Negative': -1, 'Neutral':0})
df

,Date,source,sentiment
0,2023-08-06,InvestorPlace,1
1,2023-08-06,Forbes,0
2,2023-08-06,Seeking Alpha,1
3,2023-08-06,The Motley Fool,1
4,2023-08-06,The Motley Fool,1
...,...,...,...
9895,2021-07-13,Barrons,-1
9896,2021-07-13,Yahoo Finance,1
9897,2021-07-13,The Motley Fool,1
9898,2021-07-13,Market Watch,-1


In [21]:
stock_data = pd.read_csv('/Users/manaswiniswamy/Downloads/AMZN.csv', parse_dates=True, index_col='Date')

In [22]:
stock_data['200_day_avg'] = stock_data['Close'].rolling(window=200).mean()
stock_data['50_day_avg'] = stock_data['Close'].rolling(window=50).mean()
stock_data['10_day_avg'] = stock_data['Close'].rolling(window=10).mean()

In [23]:
stock_data.dropna(inplace=True)

In [25]:
stock_data['Trend'] = np.where(stock_data['Close'].shift(-1) > stock_data['Close'], 1, 0)

In [26]:
stock_data=stock_data.merge(df,how="inner",on="Date")
stock_data

,Date,Open,High,Low,Close,Adj Close,Volume,200_day_avg,50_day_avg,10_day_avg,Trend,source,sentiment
0,2023-05-23,114.269997,117.139999,113.779999,114.989998,114.989998,67576300,105.50800,104.336200,113.713000,1,InvestorPlace,1
1,2023-05-23,114.269997,117.139999,113.779999,114.989998,114.989998,67576300,105.50800,104.336200,113.713000,1,Forbes,0
2,2023-05-23,114.269997,117.139999,113.779999,114.989998,114.989998,67576300,105.50800,104.336200,113.713000,1,Forbes,0
3,2023-05-23,114.269997,117.139999,113.779999,114.989998,114.989998,67576300,105.50800,104.336200,113.713000,1,CNN Business,-1
4,2023-05-23,114.269997,117.139999,113.779999,114.989998,114.989998,67576300,105.50800,104.336200,113.713000,1,Zacks Investment Research,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
970,2023-08-04,141.059998,143.630005,139.320007,139.570007,139.570007,152938700,105.86745,127.856601,130.860002,0,Market Watch,0
971,2023-08-04,141.059998,143.630005,139.320007,139.570007,139.570007,152938700,105.86745,127.856601,130.860002,0,WSJ,0
972,2023-08-04,141.059998,143.630005,139.320007,139.570007,139.570007,152938700,105.86745,127.856601,130.860002,0,TechXplore,1
973,2023-08-04,141.059998,143.630005,139.320007,139.570007,139.570007,152938700,105.86745,127.856601,130.860002,0,Proactive Investors,1


In [27]:
X = stock_data[['200_day_avg', '50_day_avg', '10_day_avg','sentiment']].values
y = stock_data['Trend'].values

In [28]:
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [33]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(4,)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [34]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [48]:
model.fit(X_train, y_train, epochs=2000, batch_size=32, validation_split=0.1)

Epoch 1/2000
22/22 [==============================] - 0s 2ms/step - loss: 0.3520 - accuracy: 0.8276 - val_loss: 0.4492 - val_accuracy: 0.8333
Epoch 2/2000
22/22 [==============================] - 0s 1ms/step - loss: 0.3494 - accuracy: 0.8205 - val_loss: 0.4436 - val_accuracy: 0.8590
Epoch 3/2000
22/22 [==============================] - 0s 1ms/step - loss: 0.3464 - accuracy: 0.8447 - val_loss: 0.4560 - val_accuracy: 0.8077
Epoch 4/2000
22/22 [==============================] - 0s 1ms/step - loss: 0.3486 - accuracy: 0.8291 - val_loss: 0.4579 - val_accuracy: 0.8077
Epoch 5/2000
22/22 [==============================] - 0s 1ms/step - loss: 0.3497 - accuracy: 0.8276 - val_loss: 0.4588 - val_accuracy: 0.8077
Epoch 6/2000
22/22 [==============================] - 0s 1ms/step - loss: 0.3638 - accuracy: 0.8148 - val_loss: 0.4867 - val_accuracy: 0.7564
Epoch 7/2000
22/22 [==============================] - 0s 1ms/step - loss: 0.3672 - accuracy: 0.8205 - val_loss: 0.4940 - val_accuracy: 0.7564
Epoch 

Epoch 58/2000
22/22 [==============================] - 0s 913us/step - loss: 0.3656 - accuracy: 0.8205 - val_loss: 0.4768 - val_accuracy: 0.7821
Epoch 59/2000
22/22 [==============================] - 0s 882us/step - loss: 0.3545 - accuracy: 0.8262 - val_loss: 0.4627 - val_accuracy: 0.7692
Epoch 60/2000
22/22 [==============================] - 0s 878us/step - loss: 0.3673 - accuracy: 0.8177 - val_loss: 0.4477 - val_accuracy: 0.8205
Epoch 61/2000
22/22 [==============================] - 0s 849us/step - loss: 0.3496 - accuracy: 0.8390 - val_loss: 0.4543 - val_accuracy: 0.8333
Epoch 62/2000
22/22 [==============================] - 0s 855us/step - loss: 0.3493 - accuracy: 0.8276 - val_loss: 0.4637 - val_accuracy: 0.8077
Epoch 63/2000
22/22 [==============================] - 0s 848us/step - loss: 0.3515 - accuracy: 0.8234 - val_loss: 0.4466 - val_accuracy: 0.8205
Epoch 64/2000
22/22 [==============================] - 0s 846us/step - loss: 0.3571 - accuracy: 0.8376 - val_loss: 0.4602 - val_ac

22/22 [==============================] - 0s 965us/step - loss: 0.3851 - accuracy: 0.8048 - val_loss: 0.4514 - val_accuracy: 0.8333
Epoch 115/2000
22/22 [==============================] - 0s 886us/step - loss: 0.3497 - accuracy: 0.8276 - val_loss: 0.4667 - val_accuracy: 0.7949
Epoch 116/2000
22/22 [==============================] - 0s 887us/step - loss: 0.3559 - accuracy: 0.8319 - val_loss: 0.4558 - val_accuracy: 0.7692
Epoch 117/2000
22/22 [==============================] - 0s 875us/step - loss: 0.3907 - accuracy: 0.8034 - val_loss: 0.4642 - val_accuracy: 0.7821
Epoch 118/2000
22/22 [==============================] - 0s 878us/step - loss: 0.3478 - accuracy: 0.8348 - val_loss: 0.4480 - val_accuracy: 0.8333
Epoch 119/2000
22/22 [==============================] - 0s 858us/step - loss: 0.3375 - accuracy: 0.8490 - val_loss: 0.4446 - val_accuracy: 0.8590
Epoch 120/2000
22/22 [==============================] - 0s 887us/step - loss: 0.3498 - accuracy: 0.8219 - val_loss: 0.4479 - val_accuracy: 

22/22 [==============================] - 0s 932us/step - loss: 0.3572 - accuracy: 0.8291 - val_loss: 0.4395 - val_accuracy: 0.8205
Epoch 171/2000
22/22 [==============================] - 0s 955us/step - loss: 0.3693 - accuracy: 0.8148 - val_loss: 0.4540 - val_accuracy: 0.8205
Epoch 172/2000
22/22 [==============================] - 0s 988us/step - loss: 0.3541 - accuracy: 0.8405 - val_loss: 0.4569 - val_accuracy: 0.7949
Epoch 173/2000
22/22 [==============================] - 0s 928us/step - loss: 0.3466 - accuracy: 0.8148 - val_loss: 0.4424 - val_accuracy: 0.8205
Epoch 174/2000
22/22 [==============================] - 0s 934us/step - loss: 0.3466 - accuracy: 0.8405 - val_loss: 0.4579 - val_accuracy: 0.8333
Epoch 175/2000
22/22 [==============================] - 0s 984us/step - loss: 0.3409 - accuracy: 0.8419 - val_loss: 0.4457 - val_accuracy: 0.8462
Epoch 176/2000
22/22 [==============================] - 0s 935us/step - loss: 0.3389 - accuracy: 0.8362 - val_loss: 0.4441 - val_accuracy: 

22/22 [==============================] - 0s 818us/step - loss: 0.3774 - accuracy: 0.8105 - val_loss: 0.4640 - val_accuracy: 0.7949
Epoch 227/2000
22/22 [==============================] - 0s 821us/step - loss: 0.3534 - accuracy: 0.8177 - val_loss: 0.4458 - val_accuracy: 0.8333
Epoch 228/2000
22/22 [==============================] - 0s 823us/step - loss: 0.3376 - accuracy: 0.8476 - val_loss: 0.4459 - val_accuracy: 0.8333
Epoch 229/2000
22/22 [==============================] - 0s 805us/step - loss: 0.3499 - accuracy: 0.8134 - val_loss: 0.4441 - val_accuracy: 0.8333
Epoch 230/2000
22/22 [==============================] - 0s 801us/step - loss: 0.3524 - accuracy: 0.8433 - val_loss: 0.4618 - val_accuracy: 0.7692
Epoch 231/2000
22/22 [==============================] - 0s 816us/step - loss: 0.3541 - accuracy: 0.8376 - val_loss: 0.4518 - val_accuracy: 0.7821
Epoch 232/2000
22/22 [==============================] - 0s 813us/step - loss: 0.3354 - accuracy: 0.8419 - val_loss: 0.4495 - val_accuracy: 

...

Epoch 1738/2000
22/22 [==============================] - 0s 825us/step - loss: 0.2733 - accuracy: 0.8675 - val_loss: 0.4268 - val_accuracy: 0.8333
Epoch 1739/2000
22/22 [==============================] - 0s 824us/step - loss: 0.2809 - accuracy: 0.8746 - val_loss: 0.4219 - val_accuracy: 0.8205
Epoch 1740/2000
22/22 [==============================] - 0s 829us/step - loss: 0.2824 - accuracy: 0.8604 - val_loss: 0.4369 - val_accuracy: 0.7821
Epoch 1741/2000
22/22 [==============================] - 0s 827us/step - loss: 0.2744 - accuracy: 0.8803 - val_loss: 0.4482 - val_accuracy: 0.7949
Epoch 1742/2000
22/22 [==============================] - 0s 830us/step - loss: 0.2689 - accuracy: 0.8789 - val_loss: 0.4120 - val_accuracy: 0.8462
Epoch 1743/2000
22/22 [==============================] - 0s 816us/step - loss: 0.2649 - accuracy: 0.8846 - val_loss: 0.4309 - val_accuracy: 0.8205
Epoch 1744/2000
22/22 [==============================] - 0s 808us/step - loss: 0.2801 - accuracy: 0.8632 - val_loss: 0

Epoch 1794/2000
22/22 [==============================] - 0s 820us/step - loss: 0.2668 - accuracy: 0.8860 - val_loss: 0.4144 - val_accuracy: 0.8333
Epoch 1795/2000
22/22 [==============================] - 0s 823us/step - loss: 0.2742 - accuracy: 0.8761 - val_loss: 0.4222 - val_accuracy: 0.8077
Epoch 1796/2000
22/22 [==============================] - 0s 822us/step - loss: 0.2598 - accuracy: 0.8860 - val_loss: 0.4303 - val_accuracy: 0.8077
Epoch 1797/2000
22/22 [==============================] - 0s 815us/step - loss: 0.2756 - accuracy: 0.8818 - val_loss: 0.4332 - val_accuracy: 0.8333
Epoch 1798/2000
22/22 [==============================] - 0s 831us/step - loss: 0.2742 - accuracy: 0.8618 - val_loss: 0.4205 - val_accuracy: 0.8333
Epoch 1799/2000
22/22 [==============================] - 0s 812us/step - loss: 0.2626 - accuracy: 0.8903 - val_loss: 0.4153 - val_accuracy: 0.8718
Epoch 1800/2000
22/22 [==============================] - 0s 818us/step - loss: 0.2705 - accuracy: 0.8689 - val_loss: 0

Epoch 1850/2000
22/22 [==============================] - 0s 829us/step - loss: 0.2638 - accuracy: 0.8761 - val_loss: 0.4331 - val_accuracy: 0.7949
Epoch 1851/2000
22/22 [==============================] - 0s 818us/step - loss: 0.2669 - accuracy: 0.8746 - val_loss: 0.3998 - val_accuracy: 0.8846
Epoch 1852/2000
22/22 [==============================] - 0s 817us/step - loss: 0.2634 - accuracy: 0.8818 - val_loss: 0.4071 - val_accuracy: 0.8590
Epoch 1853/2000
22/22 [==============================] - 0s 821us/step - loss: 0.2798 - accuracy: 0.8775 - val_loss: 0.4014 - val_accuracy: 0.8462
Epoch 1854/2000
22/22 [==============================] - 0s 832us/step - loss: 0.2612 - accuracy: 0.8946 - val_loss: 0.4090 - val_accuracy: 0.8462
Epoch 1855/2000
22/22 [==============================] - 0s 825us/step - loss: 0.2631 - accuracy: 0.8860 - val_loss: 0.4649 - val_accuracy: 0.7692
Epoch 1856/2000
22/22 [==============================] - 0s 808us/step - loss: 0.2678 - accuracy: 0.8875 - val_loss: 0

Epoch 1906/2000
22/22 [==============================] - 0s 828us/step - loss: 0.2736 - accuracy: 0.8761 - val_loss: 0.4353 - val_accuracy: 0.7949
Epoch 1907/2000
22/22 [==============================] - 0s 832us/step - loss: 0.2773 - accuracy: 0.8732 - val_loss: 0.5028 - val_accuracy: 0.7436
Epoch 1908/2000
22/22 [==============================] - 0s 817us/step - loss: 0.2833 - accuracy: 0.8675 - val_loss: 0.4026 - val_accuracy: 0.8462
Epoch 1909/2000
22/22 [==============================] - 0s 822us/step - loss: 0.2606 - accuracy: 0.8803 - val_loss: 0.3964 - val_accuracy: 0.8462
Epoch 1910/2000
22/22 [==============================] - 0s 815us/step - loss: 0.2634 - accuracy: 0.8803 - val_loss: 0.4462 - val_accuracy: 0.7821
Epoch 1911/2000
22/22 [==============================] - 0s 812us/step - loss: 0.2614 - accuracy: 0.8860 - val_loss: 0.4111 - val_accuracy: 0.8462
Epoch 1912/2000
22/22 [==============================] - 0s 816us/step - loss: 0.2625 - accuracy: 0.8818 - val_loss: 0

Epoch 1962/2000
22/22 [==============================] - 0s 885us/step - loss: 0.2640 - accuracy: 0.8789 - val_loss: 0.4260 - val_accuracy: 0.7949
Epoch 1963/2000
22/22 [==============================] - 0s 859us/step - loss: 0.2654 - accuracy: 0.8846 - val_loss: 0.4395 - val_accuracy: 0.7564
Epoch 1964/2000
22/22 [==============================] - 0s 841us/step - loss: 0.2611 - accuracy: 0.8846 - val_loss: 0.4051 - val_accuracy: 0.8718
Epoch 1965/2000
22/22 [==============================] - 0s 851us/step - loss: 0.2774 - accuracy: 0.8675 - val_loss: 0.4140 - val_accuracy: 0.8590
Epoch 1966/2000
22/22 [==============================] - 0s 820us/step - loss: 0.2625 - accuracy: 0.8946 - val_loss: 0.4275 - val_accuracy: 0.8333
Epoch 1967/2000
22/22 [==============================] - 0s 822us/step - loss: 0.2774 - accuracy: 0.8689 - val_loss: 0.4171 - val_accuracy: 0.8462
Epoch 1968/2000
22/22 [==============================] - 0s 821us/step - loss: 0.2677 - accuracy: 0.8704 - val_loss: 0

In [46]:
loss, accuracy = model.evaluate(X_test, y_test)

7/7 [==============================] - 0s 968us/step - loss: 0.4634 - accuracy: 0.7846


In [47]:
print(f"Test accuracy: {accuracy}")

Test accuracy: 0.7846153974533081
